In [ ]:
from PIL import Image
import sys 
# change for your path to slidebook's stuff
sys.path.insert(0, 'Python/Format7')

from SBReadFile import *

from matplotlib import pyplot as plt
import numpy as np
import napari

from dask import delayed
import dask.array as da

In [ ]:
theSBFileReader = SBReadFile()

# change for your image file
theSBFileReader.Open("sldy/128x_128y_5z_94t.sldy")

In [ ]:
def read_image_plane(capture=0, position=0, timepoint=0, zplane=0, channel=0, as_2d=True):
    return theSBFileReader.ReadImagePlaneBuf(capture, position, timepoint, zplane, channel, as_2d)

# read the first file to get the shape and dtype
# ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
sample = read_image_plane()

# true is 2D or not
capture = 0
zSize=theSBFileReader.GetNumZPlanes(capture)
cSize=theSBFileReader.GetNumChannels(capture)
tSize=theSBFileReader.GetNumTimepoints(capture)

lazy_imread = delayed(read_image_plane)  # lazy reader

In [ ]:
lazy_arrays = [[[lazy_imread(0,0,t,z,c,True) for z in range(zSize)] for t in range(tSize)] for c in range(cSize)]
dask_arrays = [
    [[da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays[c][t]] for t in range(tSize)] for c in range(cSize)
]

In [ ]:
stack = da.stack(dask_arrays)
print(stack.shape)
# in jupyter notebook the repr of a dask stack provides a useful visual:
stack

In [ ]:
napari.view_image(stack, channel_axis=0, contrast_limits=[0,255], multiscale=False)